In [1]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text

### tokenize and clean text

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

In [3]:
nlp.max_length= 1198623

In [4]:
def seprate_fun(t):
    return [token.text.lower() for token in nlp(t) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [5]:
doc = read_file('UPDATED_NLP_COURSE/06-Deep-Learning/moby_dick_four_chapters.txt')

In [6]:
tokens = seprate_fun(doc)

In [7]:
len(tokens)

11338

### Sequence of token

In [8]:
train_len = 25 + 1  # 25 words for training and last word for prediction
    
text_seq = []

for i in range(train_len , len(tokens)):
    
    seq = tokens[i-train_len : i]
    
    text_seq.append(seq)
    

In [9]:
text_seq[0]  # first sequence

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

### Keras Tokenization

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [11]:
# encode token into number of array list
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(text_seq)
sequence = tokenizer.texts_to_sequences(text_seq)

In [12]:
sequence[0]

[956,
 14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 315,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2712,
 14,
 24]

In [14]:
for i in sequence[0]:
    print(f"{i}:{tokenizer.index_word[i]}") # same sequence converted to encode numbers...

956:call
14:me
263:ishmael
51:some
261:years
408:ago
87:never
219:mind
129:how
111:long
954:precisely
260:having
50:little
43:or
38:no
315:money
7:in
23:my
546:purse
3:and
150:nothing
259:particular
6:to
2712:interest
14:me
24:on


In [15]:
len(sequence)

11312

In [ ]:
tokenizer.word_counts # how many times each word occur in given text file

In [17]:
vocab_size = len(tokenizer.word_counts)

In [18]:
vocab_size

2717

### Convert list to numpy matrix

In [19]:
import numpy as np

In [20]:
sequence_array = np.array(sequence)

In [21]:
sequence_array[0] # first sequence in nd.array format

array([ 956,   14,  263,   51,  261,  408,   87,  219,  129,  111,  954,
        260,   50,   43,   38,  315,    7,   23,  546,    3,  150,  259,
          6, 2712,   14,   24])

In [22]:
seq_len = len(sequence_array)

In [23]:
sequence_array.shape

(11312, 26)

### LSTM

In [24]:
from keras.models import Sequential
from keras.layers import Dense , LSTM , Embedding

In [25]:
def create_model(vocab_size , seq_len):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size ,output_dim = 25 ,input_length = seq_len))
    model.add(LSTM(128 , return_sequences = True))
    model.add(LSTM(128 , return_sequences = False))
    model.add(Dense(256 , activation = 'relu'))
    model.add(Dense(vocab_size , activation='softmax'))
    
    model.compile(loss='categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'])
    
    model.summary()
    
    return model

### Train Test Split

In [26]:
from keras.utils import to_categorical


In [33]:
sequence_array

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

In [34]:
sequence_array[:, :-1] # except last column

array([[ 956,   14,  263, ...,    6, 2712,   14],
       [  14,  263,   51, ..., 2712,   14,   24],
       [ 263,   51,  261, ...,   14,   24,  957],
       ...,
       [ 952,   12,  166, ...,   11,  262,   53],
       [  12,  166, 2711, ...,  262,   53,    2],
       [ 166, 2711,    3, ...,   53,    2, 2717]])

In [35]:
sequence_array[: , -1:] # last column which we have to predict

array([[  24],
       [ 957],
       [   5],
       ...,
       [   2],
       [2717],
       [  26]])

In [36]:
X = sequence_array[: , :-1]
y = sequence_array[: , -1:]

In [37]:
y = to_categorical(y , num_classes=vocab_size + 1) # we add 1 because for unknown word which index is zero

In [38]:
seq_len = X.shape[1]

### Fit the Model

In [39]:
model = create_model(vocab_size + 1 , seq_len=seq_len)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67950     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 128)           78848     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 2718)              698526    
Total params: 1,009,932
Trainable params: 1,009,932
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(X , y ,batch_size= 128,epochs = 250,verbose=1)

C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/250
11312/11312 [==============================] - 19s 2ms/step - loss: 6.8332 - accuracy: 0.0523
Epoch 2/250
11312/11312 [==============================] - 18s 2ms/step - loss: 6.3839 - accuracy: 0.0508
Epoch 3/250
11312/11312 [==============================] - 17s 1ms/step - loss: 6.3364 - accuracy: 0.0529
Epoch 4/250
11312/11312 [==============================] - 17s 2ms/step - loss: 6.1977 - accuracy: 0.0529
Epoch 5/250
11312/11312 [==============================] - 17s 2ms/step - loss: 6.1226 - accuracy: 0.0529
Epoch 6/250
11312/11312 [==============================] - 18s 2ms/step - loss: 6.0721 - accuracy: 0.0554
Epoch 7/250
11312/11312 [==============================] - 18s 2ms/step - loss: 6.0023 - accuracy: 0.0627
Epoch 8/250
11312/11312 [==============================] - 18s 2ms/step - loss: 5.8735 - accuracy: 0.0659 2s - loss: 5.8
Epoch 9/250
11312/11312 [==============================] - 16s 1ms/step - loss: 5.7691 - accuracy: 0.0700
Epoch 10/250
11312/11312 [=====

11312/11312 [==============================] - 16s 1ms/step - loss: 2.7544 - accuracy: 0.3124
Epoch 77/250
11312/11312 [==============================] - 16s 1ms/step - loss: 2.7284 - accuracy: 0.3150
Epoch 78/250
11312/11312 [==============================] - 15s 1ms/step - loss: 2.7050 - accuracy: 0.3256
Epoch 79/250
11312/11312 [==============================] - 16s 1ms/step - loss: 2.6781 - accuracy: 0.3304
Epoch 80/250
11312/11312 [==============================] - 16s 1ms/step - loss: 2.6472 - accuracy: 0.3342
Epoch 81/250
11312/11312 [==============================] - 15s 1ms/step - loss: 2.6158 - accuracy: 0.3375
Epoch 82/250
11312/11312 [==============================] - 16s 1ms/step - loss: 2.5981 - accuracy: 0.3407
Epoch 83/250
11312/11312 [==============================] - 16s 1ms/step - loss: 2.5787 - accuracy: 0.3485
Epoch 84/250
11312/11312 [==============================] - 16s 1ms/step - loss: 2.5571 - accuracy: 0.3528
Epoch 85/250
11312/11312 [========================

11312/11312 [==============================] - 16s 1ms/step - loss: 1.3623 - accuracy: 0.6362
Epoch 151/250
11312/11312 [==============================] - 16s 1ms/step - loss: 1.3486 - accuracy: 0.6414
Epoch 152/250
11312/11312 [==============================] - 16s 1ms/step - loss: 1.3376 - accuracy: 0.6414
Epoch 153/250
11312/11312 [==============================] - 16s 1ms/step - loss: 1.3210 - accuracy: 0.6508
Epoch 154/250
11312/11312 [==============================] - 16s 1ms/step - loss: 1.3088 - accuracy: 0.6514
Epoch 155/250
11312/11312 [==============================] - 16s 1ms/step - loss: 1.2896 - accuracy: 0.6578
Epoch 156/250
11312/11312 [==============================] - 16s 1ms/step - loss: 1.2792 - accuracy: 0.6612
Epoch 157/250
11312/11312 [==============================] - 17s 2ms/step - loss: 1.2674 - accuracy: 0.6634
Epoch 158/250
11312/11312 [==============================] - 17s 1ms/step - loss: 1.2723 - accuracy: 0.6565
Epoch 159/250
11312/11312 [===============

11312/11312 [==============================] - 17s 1ms/step - loss: 0.9456 - accuracy: 0.7821
Epoch 225/250
11312/11312 [==============================] - 17s 1ms/step - loss: 0.8247 - accuracy: 0.8032
Epoch 226/250
11312/11312 [==============================] - 16s 1ms/step - loss: 0.7706 - accuracy: 0.8128
Epoch 227/250
11312/11312 [==============================] - 17s 1ms/step - loss: 0.7380 - accuracy: 0.8227
Epoch 228/250
11312/11312 [==============================] - 17s 1ms/step - loss: 0.7036 - accuracy: 0.8305
Epoch 229/250
11312/11312 [==============================] - 16s 1ms/step - loss: 0.6875 - accuracy: 0.8340
Epoch 230/250
11312/11312 [==============================] - 16s 1ms/step - loss: 0.6675 - accuracy: 0.8365 0s - loss: 0.6665 - accuracy: 0.
Epoch 231/250
11312/11312 [==============================] - 17s 1ms/step - loss: 0.6511 - accuracy: 0.8386
Epoch 232/250
11312/11312 [==============================] - 17s 1ms/step - loss: 0.6536 - accuracy: 0.8371
Epoch 233

In [ ]:
from pickle import dump,load

### Genrating Text

In [41]:
from random import randint
from keras.preprocessing.sequence import pad_sequences

In [42]:
def genrate_text(tokenizer,seed_text , seq_len , model , num_gen_words):
    
    output_text = []
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text] , maxlen=seq_len , truncating='pre')
        
        pred_word_ind = model.predict_classes(pad_encoded , verbose = 0)[0]
        
        pred_word = tokenizer.index_word[pred_word_ind]
        
        input_text += ' '+pred_word
        
        output_text.append(pred_word)
        
    return ' '.join(output_text)

In [43]:
len(text_seq)

11312

In [44]:
import random

In [63]:
random.seed()
random_pick = random.randint(0 , len(text_seq))
seed_text = ' '.join(text_seq[random_pick])

In [64]:
seed_text

"again and seeing no possible chance of spending a sufferable night unless in some other person 's bed i began to think that after all i"

In [65]:
genrate_text(tokenizer=tokenizer , seed_text=seed_text , seq_len = seq_len , model=model ,num_gen_words=27)

'might be cherishing unwarrantable prejudices against this unknown harpooneer thinks i the bar room when knowing all it was a very dubious looking nay a very dark'